In [36]:
# Importando todas las bibliotecas necesarias
import pandas as pd

# Bibliotecas para el pre procesamiento del texto
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Bibliotecas para el uso de GloVe
import spacy
import numpy as np

# Bibliotecas para la construccion de modelos
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# Se descargan las stopwords y lemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10053]
[nltk_data]     Se ha anulado una conexión establecida por el software
[nltk_data]     en su equipo host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10053] Se
[nltk_data]     ha anulado una conexión establecida por el software en
[nltk_data]     su equipo host>


False

In [3]:
def preprocesar_texto(textos):
    '''
        Esta funcion permite realizar el pre procesamiento a los textos, en donde primero se eliminan 
        caracteres especiales y ajenos al alfabeto en inglés. Posteriormente se separa el texto en tokens, 
        se les aplica un proceso de lematización, se eliminan las stop words y se regresa el texto a su estado original.
    '''
    # Se inicializa el conjunto de stopwords y el lemmatizer
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Se tiene una lista en donde se almacenaran los textos despues del pre procesamiento
    textos_preprocesados = []
    
    for texto in textos:
        # Utilizando expresiones regulares se eliminan signos de puntuacion y cualquier otro caracter ajeno al ingles
        # Asimismo se pasa el texto a minusculas
        texto = re.sub(r'[^a-z\s]', '', texto.lower())

        # Se divide el texto en palabras (tokens)
        tokens = texto.split()
        
        # Se eliminan las stop words y se aplica el lemmatizer
        palabras_filtradas = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
        
        # Se vuelven a unir las palabras para conformar un mismo texto ya preprocesado
        texto_preprocesado = ' '.join(palabras_filtradas)
        
        # Se une el texto preprocesado a la lista de textos
        textos_preprocesados.append(texto_preprocesado)
    
    # Se devuelven todos los textos ya pre procesados
    return textos_preprocesados

In [4]:
# Se carga el archivo a leer
df = pd.read_csv('Suicide_Detection.csv')

# Se extraen las etiquetas y textos
textos = df['text'].values
etiquetas = df['class'].values

# Se realiza el preprocesamiento de textos
textos_preprocesados = preprocesar_texto(textos)

# Se separa el conjunto de datos en un conjunto de entrenamiento y otro de prueba
texts_train, texts_test, labels_train, labels_test = train_test_split(textos_preprocesados, etiquetas, test_size=0.2, random_state=42)

# El siguiente apartado aplica solo para los modelos de Naive Bayes y Regresion Logistica
# Se obtienen las caracteristicas de los textos usando el modelo de bolsa de palabras
vectorizer = CountVectorizer()
features_train = vectorizer.fit_transform(texts_train)
features_test = vectorizer.transform(texts_test)

# Se escalan las caracteristicas anteriores para tener los valores dentro de un intervalo de valores mas proximo
# Se hace uso de el metodo de estandarizacion
scaler = StandardScaler(with_mean=False)
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

# Modelo con Naive Bayes

In [51]:
# Se entrena el modelo utilizando el algoritmo de Naive Bayes
naive_bayes = MultinomialNB()
# Se utiliza el modelo sobre los datos y etiquetas de entrenamiento
naive_bayes.fit(features_train, labels_train)

# Se hacen predicciones con base en el conjunto de datos de prueba
predictions_naive_bayes = naive_bayes.predict(features_test)

# Se calcula la precision del modelo
accuracy = accuracy_score(labels_test, predictions_naive_bayes)
print("Accuracy:", accuracy)

Accuracy: 0.9018636216740278


# Modelo con Regresion Logística

In [52]:
# Se entrena el modelo utilizando el algoritmo de Regresion Logistica
logistic = LogisticRegression(max_iter=1000, C=0.5)
# Se utiliza el modelo sobre los datos y etiquetas de entrenamiento
logistic.fit(features_train, labels_train)

# Se hacen predicciones con base en el conjunto de datos de prueba
predictions_logistic_regresion = logistic.predict(features_test)

# Se calcula la precision del modelo
accuracy = accuracy_score(labels_test, predictions_logistic_regresion)
print("Accuracy:", accuracy)

Accuracy: 0.933900678659916


# Modelo con LSTM

In [1]:
# Este apartado es para convertir el archivo que contiene GloVe previamente procesados a un formato entendible
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

C:\Users\alexm\AppData\Local\Temp\ipykernel_6780\1287770451.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 50)

In [27]:
def obtener_GloVe(textos, padding=50, glove_file='glove.6B.50d.word2vec.txt'):
    '''
        Esta funcion permite obtener los Global Vectors (GloVe) para poder realizar la obtencion de caracteristicas
    '''
    # Se usa un conjunto previamente entrenado para GloVe con 50 dimensiones
    glove_model = KeyedVectors.load_word2vec_format('glove.6B.50d.word2vec.txt', binary=False)
    
    # Se tiene una lista en donde se van a almcenar las secuencias de vectores
    sequences = []
    
    # Se obtendran los respectivos vectores para cada uno de los textos
    for texto in textos:
        # Se obtiene el valor del vector de aquellas palabras que se encuentren en el archivo previamente entrenado
        sequence = [glove_model.key_to_index[word] for word in texto if word in glove_model.key_to_index]
        # Se adiciona el vector a la lista de vectores
        sequences.append(sequence)
    
    # Se busca que todos los vectores tengan el mismo tamaño
    padded_sequences = pad_sequences(sequences, maxlen=padding)
    
    return padded_sequences

In [40]:
# Se separa el texto en tokens para poder aplicar GloVe
tokenizer = Tokenizer()
tokenizer.fit_on_texts(textos_preprocesados)
sequences = tokenizer.texts_to_sequences(textos_preprocesados)

# Se divide el conjunto de datos para entrenamiento y pruebas
texts_train, texts_test, labels_train, labels_test = train_test_split(
    sequences, etiquetas, test_size=0.2, random_state=42
)

# Obteniendo los GloVe
glove_train = obtener_GloVe(texts_train)
glove_test = obtener_GloVe(texts_test)

# Se convierten las etiquetas a valores numericos para poder utilizar el modelo
labelEnc=LabelEncoder()
labels_trainEnc=labelEnc.fit_transform(labels_train)
labels_testEnc=labelEnc.transform(labels_test)

# Se define el modelo LSTM con tres capas: Embedding, LSTM y Dense con funcion de activacion sigmoide
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=50))
model.add(LSTM(units=128))
model.add(Dense(units=1, activation='sigmoid'))

# El modelo se compila utilizando una funcion de entropia cruzada y el optimizador con el algoritmo de Adam
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Se entrena el modelo
model.fit(glove_train, labels_trainEnc, validation_data=(glove_test, labels_testEnc), epochs=10, batch_size=64)

# Se evalua el modelo
loss, accuracy = model.evaluate(glove_test, labels_testEnc)
print("Accuracy:", accuracy)

Epoch 1/10
2901/2901 [==============================] - 691s 238ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6931 - val_accuracy: 0.5017
Epoch 2/10
2901/2901 [==============================] - 684s 236ms/step - loss: 0.6932 - accuracy: 0.4983 - val_loss: 0.6931 - val_accuracy: 0.5017
Epoch 3/10
2901/2901 [==============================] - 688s 237ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.4983
Epoch 4/10
2901/2901 [==============================] - 695s 240ms/step - loss: 0.6932 - accuracy: 0.4986 - val_loss: 0.6931 - val_accuracy: 0.5017
Epoch 5/10
2901/2901 [==============================] - 703s 242ms/step - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6932 - val_accuracy: 0.4983
Epoch 6/10
2901/2901 [==============================] - 710s 245ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6932 - val_accuracy: 0.5017
Epoch 7/10
2901/2901 [==============================] - 713s 246ms/step - loss: 0.6932 - accuracy: 0.5002 - val_

# Predicciones

In [73]:
# Se crean algunos textos de ejemplos para ver qué tal reacciona cada modelo
# Asimismo se pone la intención que tiene cada texto
texto_ejemplo1 = "I have been feeling pretty lonely lately. I don't know how to solve all of my problems, everything seems harder every day that passes. I think the only good solution for me is to end with all my suffering and end my life for good." # Intencion suicida
texto_ejemplo2 = "I have been feeling pretty lonely lately, as I broke up with my girlfriend recently and I don't know how to cope with it. Despite the fact that i want all the suffering to go away, I know that all of this is a process and someday i will get over this." # Sin intencion suicida
texto_ejemplo3 = "There are a lot of things happening in my head recently. My mom just passed away and I am having problems with my partner. I need some advices and help to go through all of this." # Sin intencion suicida
texto_ejemplo4 = "There are a lot of things happening in my head recently. My mom just passed away and I am having problems with my partner. I am having a lot of suicidal thoughts and i can't stop thinking it is the only solution." # Intencion suicida

In [74]:
# Una vez creados los textos se necesitan preprocesar para obtener mejores resultados
secuencia1 = preprocesar_texto([texto_ejemplo1])
secuencia2 = preprocesar_texto([texto_ejemplo2])
secuencia3 = preprocesar_texto([texto_ejemplo3])
secuencia4 = preprocesar_texto([texto_ejemplo4])

# Posteriormente a que fueron preprocesados se les aplica la extraccion de caracteristicas
vector_texto1 = vectorizer.transform(secuencia1)
vector_texto2 = vectorizer.transform(secuencia2)
vector_texto3 = vectorizer.transform(secuencia3)
vector_texto4 = vectorizer.transform(secuencia4)

* Naive Bayes

In [75]:
# Se realizan las predicciones de los cuatro textos
resultado1 = naive_bayes.predict(vector_texto1)
resultado2 = naive_bayes.predict(vector_texto2)
resultado3 = naive_bayes.predict(vector_texto3)
resultado4 = naive_bayes.predict(vector_texto4)
print("\tResultados:")
print(f'-->Para el primer texto se predijo la etiqueta {resultado1}')
print(f'-->Para el segundo texto se predijo la etiqueta {resultado2}')
print(f'-->Para el tercer texto se predijo la etiqueta {resultado3}')
print(f'-->Para el cuarto texto se predijo la etiqueta {resultado4}')

	Resultados:
-->Para el primer texto se predijo la etiqueta ['suicide']
-->Para el segundo texto se predijo la etiqueta ['suicide']
-->Para el tercer texto se predijo la etiqueta ['suicide']
-->Para el cuarto texto se predijo la etiqueta ['suicide']


* Regresión Logística

In [76]:
# Se realizan las predicciones de los cuatro textos
resultado1 = logistic.predict(vector_texto1)
resultado2 = logistic.predict(vector_texto2)
resultado3 = logistic.predict(vector_texto3)
resultado4 = logistic.predict(vector_texto4)
print("\tResultados:")
print(f'-->Para el primer texto se predijo la etiqueta {resultado1}')
print(f'-->Para el segundo texto se predijo la etiqueta {resultado2}')
print(f'-->Para el tercer texto se predijo la etiqueta {resultado3}')
print(f'-->Para el cuarto texto se predijo la etiqueta {resultado4}')

	Resultados:
-->Para el primer texto se predijo la etiqueta ['suicide']
-->Para el segundo texto se predijo la etiqueta ['non-suicide']
-->Para el tercer texto se predijo la etiqueta ['non-suicide']
-->Para el cuarto texto se predijo la etiqueta ['suicide']


* LSTM

In [77]:
# Posteriormente a que fueron preprocesados se les aplica la extraccion de caracteristicas con GloVe 
texto_tokens = tokenizer.texts_to_sequences([secuencia1, secuencia2, secuencia3, secuencia4])
glove_textos = obtener_GloVe(texto_tokens)

In [78]:
resultados = model.predict(glove_textos)

1/1 [==============================] - 0s 450ms/step


In [84]:
print(vector_texto4.size)

16


> En este caso los valores mayores a 0.5 se toman como un 1, lo cual significa que la etiqueta predicha es 'suicide'